Our method enables collision avoidance for convex sets by enforcement of CBF constraints between state-dependent convex sets (defined as smooth and strongly convex maps).

In particular, our method allows for ellipsoid-polytope collision avoidance and fast computation of the minimum distance between convex sets and the distance derivative. The distance derivative can then be used to enforce CBF constraints.

This repository provides:
- Geometry classes to define state-dependent strongly convex sets (smooth and strongly convex maps).
- Ipopt solver interface to compute the minimum distance between strongly convex maps.
- An ODE solver (the KKT solution ODE) that rapidly computes the minimum distance and KKT solutions along state trajectories. The KKT solutions can be used to compute the distance derivative and thus enforce CBF constraints.

In the following, we define smooth and strongly convex maps, show how to define such maps, and compute the minimum distance between them (see the [paper](https://github.com/HybridRobotics/cbf-convex-maps/tree/main) for full details).

---

### Smooth and strongly convex maps

A *Smooth convex map* is defined by a set-valued map
$\mathcal{C}: \mathcal{X} \rightarrow 2^{\mathbb{R}^l}$,
where $\mathcal{X}$ is the state space of the system and $l$ is the dimension of the convex set.
So, the convex set at state $x \in \mathcal{X}$ is given by $\mathcal{C}(x)$.
$\mathcal{C}$ is assumed to have the following form:

\begin{equation*}
\mathcal{C}(x) = \{z \in \mathbb{R}^l: f_k(x, z) \leq 0, \ \forall k \in \{1, ..., r\}\},
\end{equation*}
where $f_k: \mathcal{X} \times \mathbb{R}^l \rightarrow \mathbb{R}$, and $r$ is the number of constraints used to define $\mathcal{C}(x)$.

Smooth convex maps must satisfy the following assumptions:
- $f_k$ is twice continuously differentiable on $\mathcal{X} \times \mathbb{R}^l$ $\forall k \in \{1, ..., r\}$.
- The set $\mathcal{C}(x)$ satisfies linear independence constraint qualification (LICQ), i.e., the set of gradients of active constraints, $\{\nabla_{z} f_k(x,z): f_k(x, z) = 0\}$ is linearly independent for all $z \in \mathcal{C}(x)$ and $x \in \mathcal{X}$.
- For all $x \in \mathcal{X}$, $\mathcal{C}(x)$ has a non-empty interior.

Smooth convex maps include the set of polyhedra in $\mathbb{R}^3$ such that exactly three halfplanes define each vertex (if any).

*Strongly convex maps* are smooth convex maps with the additional propoerty that $\mathcal{C}(x)$ is compact at each $x \in \mathcal{X}$ and $\nabla_{z}^2 f_k(x, z) \succ 0$, $\forall k \in \{1, ..., r\}$, $z \in \mathbb{R}^l$, and $x \in \mathcal{X}$.

Strongly convex maps include ellipsoids and softmax approximations of bounded polyhedra.
Our method works when at least one of the two smooth convex maps considered is strongly convex.

---

### Defining smooth and strongly convex maps in code

To define convex maps, we need to provide the vector function $f(x, z) := (f_1, ..., f_r)(x, z) \in \mathbb{R}^r$ and its derivatives.
Each convex map inherits from the abstract class `ConvexSet` defined in `include/sccbf/geometry/` and implements all the required virtual functions.

The virtual function `UpdateDerivative` requires the functions (for some $\dot{x}$ and $y \in \mathbb{R}^r$):
- $f(x, z) \in \mathbb{R}^r$,
- $D_x f(x, z)[\dot{x}] \in \mathbb{R}^r$,
- $D_z f(x, z) \in \mathbb{R}^{r \times l}$,
- $\nabla_z (y^\top D_x f(x, \cdot)[\dot{x}])(z) \in \mathbb{R}^l$,
- $\nabla_z^2 (y^\top f(x, \cdot))(z) \in \mathbb{R}^{l \times l}$, and
- (optional, only required when `use_kkt_err_tol = false` (see next section)) $M(x) \in \mathbb{R}^{l \times l}$ such that $\nabla_z^2 (y^\top f(x, \cdot))(z) \succeq M(x)$ for all $z$.

The virtual function `LieDerivatives` require the lie derivatives $L_{f_d} (y^\top f(\cdot, z))(x)$ and $L_{g_d} (y^\top f(\cdot, z))(x)$ where $f_d$ and $g_d$ define the dynamics of the system.

Finally, the virtual function `get_projection_matrix` can be used to project convex sets to lower dimensions (see the paper).

- A rigid polytope in $\mathbb{R}^3$ with states in $x = (p, R) \in SE(3)$ can be defined as a smooth convex map:
\begin{equation*}
\mathcal{C}(x) = \{z: A(R^\top (z - p)) \leq b \}.
\end{equation*}
See the class `Polytope` and its definition in `src/geometry/polytope.cc`.

- Similarly, a rigid ellipsoid in $\mathbb{R}^3$ with states in $SE(3)$ can be defined as a strongly convex map (the virtual function `is_strongly_convex` must return true for strongly convex maps):
\begin{equation*}
\mathcal{C}(x) = \{z: (R^\top (z - p))^\top Q(R^\top (z - p)) \leq 1 \}.
\end{equation*}
See the class `Ellipsoid` and its definition in `src/geometry/ellipsoid.cc`.

The classes `MinkowskiSumSet` and `IntersectionSet` in `include/sccbf/transformation/` can be used for Minkowski sums and intersections of convex maps (see the paper).

---

### Computing the minimum distance, KKT solutions, and the distance derivatives

See `apps/kkt_ode_example/kkt_ode.cc` for a complete example.

After defining the convex maps, we define an Ipopt distance solver (for initializing the minimum distance and KKT solution):
```
auto solver = std::make_shared<DistanceSolver>();
```

The solver options for the KKT solution ODE solver are defined next:
```
MatrixXd metric = MatrixXd::Identity(3, 3);
auto opt = std::make_shared<SolverOptions>();
opt->metric = metric;
opt->kkt_ode.use_kkt_err_tol = true;
opt->kkt_ode.timestep = dt;
```
The matrix `metric` allows for general distance metrics given by `(z1 - z2).transpose() * metric * (z1 - z2)`.
The KKT solution ODE solver has two tolerance metrics for the KKT error at any time: `max_inf_kkt_err` and `max_primal_dual_gap`.
When the KKT error (defined using either of the two metrics) exceeds the corresponding tolerance, the Ipopt solver is used to correct the KKT solution.
For definitions of these error metrics, see `CollisionPair::KktError` and `CollisionPair::PrimalDualGap_` in `src/collision/collision_pair.cc`.

Finally, we define a collision pair using the shared pointers of the two convex maps:
```
auto cp = std::make_shared<CollisionPair>(c1_ptr, c2_ptr, opt, solver);
```

Before computing the minimum distance or using the KKT solution ODE solver to update KKT solutions, we must update the states of the convex sets.
```
cp->get_set1()->set_states(x1, dx1);
cp->get_set2()->set_states(x2, dx2);
```

The minimum distance can be computed using the Ipopt solver by calling `cp->MinimumDistance();`.

After setting the states of both convex sets, the KKT solution can be updated for time step `dt` by calling `cp->KktStep();`.

The derivative of the minimum distance (at the states and state derivatives provided) can be computed by calling `cp->GetMinimumDistanceDerivative();`.

---

### Enforcement of CBF constraints

See `apps/ma_cbf_example/controller.h` for details on enforcing the CBF constraint.

The CBF constraint on the minimum distance $h(x) = h(x_1, x_2)$ is given by
\begin{equation*}
\dot{h}(x, u) \geq -\alpha_\text{CBF} \cdot h(x),
\end{equation*}
where $\dot{h}$ is computed along $\dot{x}$ given by the input $u$.

This constraint is enforced as (see the paper):
\begin{equation*}
\lambda^*(x)^\top D_x f(x, z^*(x)) (f_d(x) + g_d(x)u) \geq -\alpha_\text{CBF} \cdot h(x),
\end{equation*}
where $(z^*(x), \lambda^*(x))$ is the optimal KKT solution at $x$.

First, a matrix of the dynamics $f_d(x)$ and $g_d(x)$ is defined for both the systems for each convex set:
```
MatrixXd fg1(nx1, nu1 + 1), fg2(nx2, nu2 + 1); // Provide definitions
```
Then, we define empty matrices to store the value of $\lambda^*(x)^\top D_x f(x, z^*(x)) [f_d(x) \ g_d(x)]$:
```
MatrixXd L_fg1 = MatrixXd::Zero(1, nu1 + 1);
MatrixXd L_fg2 = MatrixXd::Zero(1, nu2 + 1);
```

Then, we use the collision pair `cp` to update the Lie derivatives:
```
cp->LieDerivatives(fg1, fg2, L_fg1, L_fg2);
```

Finally, we can use the Lie derivatives to enforce the CBF:
```
double h = cp->get_minimum_distance();

double L_f1 = L_fg1(0, 0);
double L_f2 = L_fg2(0, 0);
MatrixXd L_g1 = L_fg1.rightCols(nu1);
MatrixXd L_g2 = L_fg2.rightCols(nu2);

// Symbolic CBF constraint
L_f1 + L_f2 + L_g1 * u1 + L_g2 * u2 >= -alpha_cbf * h;
```

---

### Debugging the code

We provide a few methods to debug issues in the code.

First, the functions `NumericalDerivatives` and `NumericalLieDerivatives` should be used to check the implementation of the function $f$ and its derivatives.
See the test functions in `tests/geometry/geometry_test.cc`.

The dynamics functions $f_d$ and $g_d$ can also be checked to see if the `IntegrateDynamics` function for the dynamical system is correct.
The dynamics check can be performed using the function `NumericalDynamics`; see the test functions in `tests/system/system_test.cc`.

All the numerical derivative functions are implemented in `src/utils/numerical_derivatives.cc`.

Second, the effect of the KKT solution ODE solver on the CBF implementation can be isolated.
This can be done by disabling the KKT solution ODE solver and directly using the Ipopt solver to compute the minimum distances (note that this will result in a significantly longer computation time).
To disable the KKT solution ODE, we can set the error tolerances for the KKT solution ODE solver in `SolverOptions` to zero:
```
auto opt = std::make_shared<SolverOptions>();
opt->max_inf_kkt_err = 0.0;
opt->max_primal_dual_gap = 0.0;
```